In [2]:
#Enter the following details to start the analysis.

#************************************************
# Enter the working directory in the following format 
#(Copy paste the address path to the directory with double slash inbetween)
# path = "C:\\Users\Dhivyabharathi\\OneDrive - Trinity College Dublin\mywork\\Python scripts\\Dashboard"
path = "C:\\Users\\ctrans\\OneDrive - iitr.ac.in\\Dhivya_Research\\Python_Scripts"
#***********************************************

#Enter the dates
#Date range should not be more than seven days
start_date='2019-12-01'
End_date='2019-12-02'
#***********************************************

#Enter the counter Id
location=1506 
#***********************************************

#Enter the direction of traffic as South bound or North boun or West bound or East bound
direction='South bound' 
#***********************************************

#Enter the speedlimit 
vf=120 #kmph
#***********************************************

#Saving variables to directory
import os
os.chdir(path)
import pickle
with open('dash_variables','wb') as f:
    pickle.dump([path, location, start_date, End_date, direction, vf],f)
    
print('Inputs are saved. Run the next block')

Inputs are saved. Run the next block


In [3]:
#Function to create the url for downloading data set
import pandas as pd
import datetime

def url(t):
    format2=t.strftime("%Y-%m-%d")
    format1=t.strftime("%Y/%m/%d")
    filename='https://data.tii.ie/Datasets/TrafficCountData/'+ format1 + '/per-vehicle-records-' + format2 +'.csv'
    return filename

#download selected days data and write to csv
def download(start_date, End_date):
    temp1=pd.date_range(start_date,End_date, freq='D') 
    for i in range(0,temp1.size):
        data = pd.read_csv(url(temp1[i]))
        tempname=temp1[i].strftime("%Y-%m-%d")+ '.csv'
        data.to_csv(tempname) 
        
# clearing temporary variables
def clear(variables):
    for var in variables:
        del var
        
#removing unwanted columns and formatting datetime       
def process(data):
    data['Datetime']=data.year.astype(str)+ '-' +data.month.astype(str)+'-'+data.day.astype(str)+ ' ' + data.hour.astype(str)+ ':' + data.minute.astype(str) + ':' + data.second.astype(str)
    data['Datetime']=pd.to_datetime(data['Datetime'].astype(str), format='%Y-%m-%d %H:%M:%S')
    data1 = data[['Datetime', 'cosit' ,'classname', 'lanename', 'speed', 'headway']].copy()
#     data1['lanename'] = data1['lanename'].str.replace('\d+', '')
    data1.set_index('Datetime', inplace=True)
    return data1

#splitting directions of traffic
def direction(data3):
    if 'Southbound 2' in data3['lanename'].unique():
        searchfor_s=['Southbound ' + str(i + 1) for i in range(0,4)]
    elif 'Southbound' in data3['lanename'].unique():
        searchfor_s = ['Southbound']
    elif 'Westbound' in data3['lanename'].unique():
        searchfor_s = ['Westbound']
    elif 'Westbound 2' in data3['lanename'].unique():
        searchfor_s=['Westbound ' + str(i + 1) for i in range(0,4)]
    else:
        searchfor_s=[]
    return searchfor_s

#Dowload the counter data 
os.chdir(r'.\Processedfiles')
download(start_date, End_date)
filelist=os.listdir(path)
print('Download completed')
# filelist

# #Vehicle composition Estimation
# veh_composition=result.groupby('classname')['lanename'].count().reset_index()
# veh_composition['Percentage'] = 100 * veh_composition['lanename']  / veh_composition['lanename'].sum()
# veh_composition.set_index('classname', inplace=True)

# os.chdir(path)
# import pickle
# filename = 'veh_composition'
# outfile = open(filename,'wb')
# pickle.dump(veh_composition,outfile)
# outfile.close()

C:\Users\ctrans\AppData\Local\Temp\ipykernel_18876\3273844419.py:15: DtypeWarning: Columns (12,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url(temp1[i]))
C:\Users\ctrans\AppData\Local\Temp\ipykernel_18876\3273844419.py:15: DtypeWarning: Columns (12,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url(temp1[i]))


Download completed


In [ ]:
#Data processing block 1
print('Data is getting prepared')
dfseries=[]
result=pd.DataFrame()
data=pd.DataFrame()
for file in filelist:
    data =pd.read_csv(file) #reading a csv
    data2=process(data) #removing unwanted columns and formatting datetime
    data3=data2[(data2['cosit']==location)]
    searchfor=direction(data3)
    data4=data3[data3['lanename'].str.contains('|'.join(searchfor))]
    dfseries.append(data4)
    clear(['data', 'data2', 'data3', 'data4'])
    result= pd.concat(dfseries)
    clear(['dfseries'])
print('Data preparation completed')

In [ ]:
#Data processing block 2
#Estimation of aggregated flow and speed
data=result
flow=[]
speed=[]
flow.append(data['classname'].resample('H').count()) #Flow is aggregated hourly
speed.append(data['speed'].resample('H').mean()) #Speed is aggregated hourly
merged_flow = pd.concat(flow)
merged_speed = pd.concat(speed)

#database preparation
merged=pd.concat([merged_flow, merged_speed], axis=1)
merged.columns =['HourlyFlow','meanspeed']

os.chdir(r'.\Processedfiles')
merged.to_csv('Output.csv')
print('Data Processing completed')
print('Results are saved into the Processedfiles directory')
merged